## Homework 04

### Exercise 1

Let $\theta_1$ and $\theta_2$ be real valued parameters in $[0,1]$ and consider the generative model
\begin{align*}
\theta_1 &\sim \theta_1\text{-prior}\\
\theta_2 &\sim \theta_2\text{-prior}\\
\hat{y} &= \frac{\theta_1+x^2}{\theta_2\cdot x}\\
y &\sim \mathcal{N} (\hat{y}, 1)
\end{align*}

a. Use pyro to implement the model as a function `model(theta1_prior, theta2_prior, x, obs)`, where `theta1_prior` and `theta2_prior` are pyro.distributions objects, `x` and `obs` are torch tensors, and draws from the normal distribution are conditioned on `obs`.

b. Choose two suitable prior distributions for $\theta_1$ and $\theta_2$ (e.g. suitably rescaled Normal or Beta distributions)  and use HMC or NUTS algorithm to find their posterior distributions given the observations

\begin{align*}
x&=(47,87,20,16,38,5)\\
y&=(58.76, 108.75,  25.03,  20.03,  47.51,  6.37).
\end{align*}


c. Discuss how different prior distributions lead to different estimates of $\theta_1$ and $\theta_2$. Comment on the convergence checks and plot the posterior distributions. 

In [7]:
import pyro
import torch
import numpy as np
import pyro.distributions as dist
from pyro.infer.mcmc import MCMC, NUTS

def model(theta1_prior, theta2_prior, x, obs):
    theta_1 = pyro.sample('theta_1', theta1_prior)
    theta_2 = pyro.sample('theta_2', theta2_prior)
    y = (theta_1+x**2)/(theta_2*x)
    return pyro.sample('y', dist.Normal(y,1), obs=obs)

x = torch.tensor([47,87,20,16,38,5])
y = torch.tensor([26.1407, 48.3493, 11.1806,  8.9757, 21.1477,  3.0556])

In [63]:
nuts_kernel = NUTS(model)
mcmc = MCMC(nuts_kernel, warmup_steps=5000, num_samples=1000, num_chains=1)
mcmc.run(theta1_prior=dist.Beta(5,1), theta2_prior=dist.Beta(2,0.5), x=x, obs=y)
mcmc.summary()

Sample: 100%|██████████| 6000/6000 [00:54, 110.30it/s, step size=5.50e-01, acc. prob=0.901]


                mean       std    median      5.0%     95.0%     n_eff     r_hat
   theta_1      0.76      0.18      0.81      0.48      1.00    521.79      1.00
   theta_2      1.00      0.00      1.00      1.00      1.00    758.63      1.00

Number of divergences: 0


### Exercise 2

A bivariate Gibbs sampler for a vector $x=(x_1,x_2)$ draws iteratively from the posterior conditional distributions in the following way:
- choose a starting value $p(x_1|x_2^{(0)})$
- for each iteration $i$:
    - draw $x_2(i)$ from $p(x_2|x_1^{(i-1)})$
    - draw $x_1(i)$ from $p(x_1|x_2^{(i)})$

a. Supposing that samples are drawn from a bivariate normal distribution

$$
{x_1 \choose x_2} \sim \mathcal{N} \Bigg[ {0 \choose 0} , \begin{pmatrix} 1 & \rho \\ \rho & 1 \end{pmatrix} \Bigg],
$$
    implement a Gibbs sampler function which takes as inputs the parameter `rho`, the number of iterations `iters` and the number of warmup draws `warmup`.



In [105]:
def gibbs_sampler(x1_init, x2_init, rho, iters, warmup):
    x1 = torch.zeros(warmup+iters)
    x2 = torch.zeros(warmup+iters)
    
    x1[0] = x1_init
    x2[0] = x2_init
    
    for i in range(1, warmup+iters):
        #x1[i] = pyro.sample('x1', dist.Normal(rho*x2[i-1].item(), np.sqrt(1-rho**2)))
        #x2[i] = pyro.sample('x2', dist.Normal(rho*x1[i-1].item(), np.sqrt(1-rho**2)))
        x1[i] = rho*x2[i-1].item()+np.sqrt(1-rho**2)*pyro.sample('x1', dist.Normal(0, 1))
        x2[i] = rho*x1[i-1].item()+np.sqrt(1-rho**2)*pyro.sample('x2', dist.Normal(0, 1))
        
    x1 = x1[warmup:]
    x2 = x2[warmup:]
    
    return x1, x2

b. Use your implementation of Gibbs sampler to infer the parameters $\theta=(\theta_1,\theta_2)$ from **Exercise 1**.

In [106]:
theta1_samples = mcmc.get_samples()['theta_1']
theta2_samples = mcmc.get_samples()['theta_2']
sigma = np.corrcoef([theta1_samples.numpy(), theta2_samples.numpy()])
estimated_rho = sigma[0,1]
estimated_rho

-0.042326237284318134

In [108]:
theta1, theta2 = gibbs_sampler(x1_init=theta1_samples.mean(), x2_init=theta2_samples.mean(), rho=estimated_rho, 
                               iters=1000, warmup=10000)

print(theta1.mean(), theta2.mean())

tensor(0.0199) tensor(-0.0104)


In [ ]:

def conditional_sampler(sampling_index, current_x, mean, cov):
    conditioned_index = 1 - sampling_index 
    # The above line works because we only have 2 variables, x_0 & x_1
    a = cov[sampling_index, sampling_index]
    b = cov[sampling_index, conditioned_index]
    c = cov[conditioned_index, conditioned_index]

    mu = mean[sampling_index] + (b * (current_x[conditioned_index] - mean[conditioned_index]))/c
    sigma = np.sqrt(a-(b**2)/c)
    new_x = np.copy(current_x)
    new_x[sampling_index] = np.random.randn()*sigma + mu
    return new_x

def gibbs_sampler(initial_point, num_samples, mean, cov):

    point = np.array(initial_point)
    samples = np.empty([num_samples+1, 2])  #sampled points
    samples[0] = point
    tmp_points = np.empty([num_samples, 2]) #inbetween points

    for i in range(num_samples):
        # Sample from p(x_0|x_1)
        point = conditional_sampler(0, point, mean, cov)
        tmp_points[i] = point
        # Sample from p(x_1|x_0)
        point = conditional_sampler(1, point, mean, cov)
        samples[i+1] = point

    return samples, tmp_points

print(theta1_samples.mean(), theta2_samples.mean())

theta1, theta2 = gibbs_sampler(initial_point=[theta1_samples.mean(), theta2_samples.mean()], 
                               cov=sigma, num_samples=10000, mean=[theta1_samples.mean(), theta2_samples.mean()])

print(theta1.mean(), theta2.mean())

tensor(0.7639) tensor(0.9999)
